<a href="https://colab.research.google.com/github/EthanCui2008/Kaggle/blob/Main/Robot_Gym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gymnasium
import numpy as np
from gymnasium import spaces
class ContinuousXYRobotEnv(gymnasium.Env):
    def __init__(self):
        super(ContinuousXYRobotEnv, self).__init__()

        self.GYM_SIZE = (27, 10)  # Size of the gym (width, height)
        self.ROBOT_SIZE = (0.5, 0.5)  # Size of the robot (width, height)
        self.ROBOT_MASS = 1.0  # Mass of the robot

        # Define action and observation space
        # Actions: [move_x, move_y, pick_up/drop]
        self.action_space = spaces.Box(low=np.array([-1, -1, 0]), high=np.array([1, 1, 1]), dtype=np.float32)

        # Observations: [robot_x, robot_y, object_x, object_y, goal_x, goal_y, holding_object]
        self.observation_space = spaces.Box(low=0, high=10, shape=(7,), dtype=np.float32)

        self.seed()
        self.reset()

    def seed(self, seed=None):
        self.np_random, _ = gym.utils.seeding.np_random(seed)

    def reset(self, seed=None, options=None):
        if seed is not None:
            self.seed(seed)
        self.robot_pos = np.array([5.0, 5.0], dtype=np.float32)
        self.object_pos = np.array([self.np_random.uniform(0, 10), self.np_random.uniform(0, 10)], dtype=np.float32)
        self.goal_pos = np.array([self.np_random.uniform(0, 10), self.np_random.uniform(0, 10)], dtype=np.float32)
        self.holding_object = 0
        return self._get_obs(), {}

    def _get_obs(self):
        return np.concatenate([self.robot_pos, self.object_pos, self.goal_pos, [self.holding_object]]).astype(np.float32)

    def step(self, action):
        move_x, move_y, pick_up = action

        # Update robot position with continuous values
        self.robot_pos[0] = np.clip(self.robot_pos[0] + move_x, 0, 10)
        self.robot_pos[1] = np.clip(self.robot_pos[1] + move_y, 0, 10)

        # Reward for moving closer to the object
        distance_to_object = np.linalg.norm(self.robot_pos - self.object_pos)
        reward = 1.0 / (distance_to_object + 1)

        # Picking up or dropping the object
        if pick_up > 0.5 and np.linalg.norm(self.robot_pos - self.object_pos) < 0.5 and not self.holding_object:
            self.holding_object = 1
            reward += 10  # Reward for picking up the object
        elif pick_up > 0.5 and np.linalg.norm(self.robot_pos - self.goal_pos) < 0.5 and self.holding_object:
            self.holding_object = 0
            reward += 50  # Reward for delivering the object to the goal

        # Check if episode is done
        done = False
        if not self.holding_object and np.linalg.norm(self.robot_pos - self.goal_pos) < 0.5:
            done = True

        return self._get_obs(), reward, done, False, {}

    def render(self, mode='human'):
        print(f"Robot Position: {self.robot_pos}")
        print(f"Object Position: {self.object_pos}")
        print(f"Goal Position: {self.goal_pos}")
        print(f"Holding Object: {self.holding_object}")


In [ ]:
env = ContinuousXYRobotEnv()

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy

# Check the environment
check_env(env)

# Create the PPO model
model = PPO("MlpPolicy", env, verbose=1)

# Train the agent
model.learn(total_timesteps=100000)

# Save the model
model.save("ppo_custom_env")

# Load the trained model
model = PPO.load("ppo_custom_env")

# Evaluate the policy
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward} +/- {std_reward}")

# Render the result after training
obs, _ = env.reset()
done = False
while not done:
    action, _states = model.predict(obs)
    obs, reward, done, _, _ = env.step(action)
    env.render()
    print(f"Reward: {reward}")

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/env_checker.py:453: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 270      |
|    ep_rew_mean     | 37.3     |
| time/              |          |
|    fps             | 1251     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 265         |
|    ep_rew_mean          | 70.4        |
| time/                   |             |
|    fps                  | 351         |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010501184 |
|    clip_fraction        | 0.122       |
|    clip_range           | 0.2         |
|    entropy_loss   

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
!pip install stable_baselines3

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 9.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl